In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [1]:
pip install unsloth

  Using cached unsloth-2026.1.4-py3-none-any.whl.metadata (66 kB)
  Using cached unsloth_zoo-2026.1.4-py3-none-any.whl.metadata (32 kB)
  Using cached torch-2.10.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (31 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached tyro-1.0.5-py3-none-any.whl.metadata (12 kB)
  Using cached xformers-0.0.34-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached triton-3.6.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
  Using cached datasets-4.3.0-py3-none-any.whl.metadata (18 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.18.1-py3-none-any.whl.metadata (14 kB)
  Using cached huggingface_hub-1.3.5-py3-none-any.whl.metadata (13 kB)
  Using cached h

In [1]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", 
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

 
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/u/philo/.conda/envs/unsloth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Qwen3 patching. Transformers: 4.57.6.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.495 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:17<00:00,  5.68s/it]


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16, 
    lora_dropout = 0.05, 
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = True, 
    loftq_config = None,
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.4 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [3]:
from datasets import load_dataset

data_path = "t3_sft_kepler_prompt.jsonl"
ds = load_dataset("json", data_files=data_path, split="train")

print(ds)
print(ds[0]["messages"])

Dataset({
    features: ['messages'],
    num_rows: 207
})
[{'role': 'user', 'content': '\nYou are annotating classroom discussion transcripts.\n\nTask:\nDecide whether the following utterance involves *proposing strategies or plans*.\n\nContext: \nStudents are working in groups on activities to learn about kepler\'s first law of planetary motion. There is a pen and paper activity (that uses pins, paper, pencil, string) for them to understand how draw an elliptical orbit and then a computer aspect where they work on various immersive computer simulation activities to develop a final claim that orbits are elliptical. The learning objective is for them to work collaboratively to discover this new knowledge through hands on activities.\n\nDefinition:\n- Articulating specific steps, strategies, or procedures required to organize or accomplish the group\'s task.\n- Look for utterances that set direction or specify how to complete an activity (often using procedural or sequential language). 

In [4]:
def to_text(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False,  # 因为 messages 里已经有 assistant 的答案
    )
    return {"text": text}

ds_text = ds.map(to_text, remove_columns=ds.column_names)

print(ds_text[0]["text"][:800])


<|im_start|>user

You are annotating classroom discussion transcripts.

Task:
Decide whether the following utterance involves *proposing strategies or plans*.

Context: 
Students are working in groups on activities to learn about kepler's first law of planetary motion. There is a pen and paper activity (that uses pins, paper, pencil, string) for them to understand how draw an elliptical orbit and then a computer aspect where they work on various immersive computer simulation activities to develop a final claim that orbits are elliptical. The learning objective is for them to work collaboratively to discover this new knowledge through hands on activities.

Definition:
- Articulating specific steps, strategies, or procedures required to organize or accomplish the group's task.
- Look for utt


In [5]:
ds_split = ds_text.train_test_split(test_size=0.1, seed=42)
train_ds = ds_split["train"]
val_ds   = ds_split["test"]

print("train:", len(train_ds), "val:", len(val_ds))


train: 186 val: 21


In [7]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,   # 必须有 "text" 列
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        max_seq_length = 2048,

        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,   # ✅ 小数据更稳一点（等效 batch=16）
        warmup_steps = 5,

        num_train_epochs = 3,              # ✅ 用 epoch 更可控
        # max_steps = 50,                  # 选 steps 才开这个；epochs 和 steps 二选一

        learning_rate = 1e-4,              # ✅ 小数据更稳（你也可以继续 2e-4）
        logging_steps = 1,

        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,

        output_dir = "qwen_lora_kepler",
        save_steps = 25,
        save_total_limit = 2,

        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=64): 100%|██████████| 186/186 [00:09<00:00, 20.38 examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [8]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 186 | Num Epochs = 3 | Total steps = 36
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 64,225,280 of 14,832,532,480 (0.43% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.653100
2,3.670700
3,3.626200
4,3.522100
5,3.251800
6,2.951700
7,2.620500
8,2.342500
9,2.097300
10,1.878900


In [10]:
trainer.model.save_pretrained("qwen_lora_kepler_adapter")
tokenizer.save_pretrained("qwen_lora_kepler_adapter")
print("saved adapter -> qwen_lora_kepler_adapter")

saved adapter -> qwen_lora_kepler_adapter
